# How to Configure a Thread

There are two attributes of the thread that we can configure, they are:
- **Thread name**: the named used to identify the thread
- **Thread daemon**: whether the thread is a background thread (daemon) or not.

Setting a unique or meaningful thread name is helpful when debugging and logging to differentiate it from other possible threads of execution.

Example:

In [1]:
import threading
from threading import Thread, Lock, RLock, Event, Condition, current_thread
from time import sleep

In [27]:
thread = Thread(name='MyThread', daemon=True)

print(thread.name)
print(thread.daemon)

MyThread
True


# How to Use Daemon Threads

A daemon thread is a background thread.

Use of daemon threads might include:
- Background logging to file or database.
- Background data retrieval, updates, refresh.
- Background data storage to disk or database.

In [28]:
def task():
    # get the current thread
    thread = current_thread()
    print(f"Daemon thread: {thread.daemon}")

thread = Thread(target=task, daemon=True)
thread.start()
thread.join()

Daemon thread: True


## Change a Thread to be a Daemon

In [29]:
def task():
    thread = current_thread()
    print(f"Daemon thread: {thread.daemon}")
    
thread = Thread(target=task)
thread.daemon = True
thread.start()
thread.join()

Daemon thread: True


## Create a Daemon Thread from a Daemon Thread

In [30]:
def task2():
    thread = current_thread()
    print(f"Daemon thread 2: {thread.daemon}")
    
def task():
    thread = current_thread()
    print(f"Daemon thread 1: {thread.daemon}")
    
    # The daemon property of the new thread will inherit from the current thread
    new_thread = Thread(target=task2)
    new_thread.start()
    new_thread.join()
    
thread = Thread(target=task, daemon=True)
thread.start()
thread.join()

Daemon thread 1: True
Daemon thread 2: True


## Terminate Daemon Threads Abruptly

Python process will exit once all non-daemon threads have terminated. Daemon threads are abruptly stopped at shutdown. Their resources may not be released properly.

In [3]:
# function to be executed in a new thread
def daemon_task():
    # get the current thread
    thread = current_thread()
    # report if daemon thread
    print(f'Daemon thread: {thread.daemon}')
    # loop for a while
    for i in range(1000):
        print(i)
        # block for a moment
        sleep(0.1)
 
def task():
    # create a new daemon thread
    daemon_thread = Thread(target=daemon_task, daemon=True)
    # start the new thread
    daemon_thread.start()
    # block for a moment to let the daemon thread run
    sleep(3)
    # prepare the user
    print('Worker thread exiting...')

thread = Thread(target=task)

# Don't run the following two lines of code
# thread.start()
# thread.join()

## Handle Unexpected Exceptions in Threads

In [4]:
def custom_hook(args):
    print(f"Thread failed: {args.exc_value}")

def task():
    print("Working...")
    sleep(1)
    raise Exception("Something went bad.")

oldhook = threading.excepthook
threading.excepthook = custom_hook
thread = Thread(target=task)
thread.start()
thread.join()
threading.excepthook = oldhook

Working...


Exception in thread Thread-6 (task):
Traceback (most recent call last):
  File "C:\Users\iwewa\.pyenv\pyenv-win\versions\3.11.3\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\iwewa\.pyenv\pyenv-win\versions\3.11.3\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\iwewa\AppData\Local\Temp\ipykernel_11236\3635390440.py", line 7, in task
Exception: Something went bad.


In [3]:
threading.excepthook

<function _thread._excepthook>

## Blocking Calls

In [3]:
lock = Lock()
with lock:
    print("lock acquired")
print("lock released")

lock acquired
lock released


In [5]:
condition = Condition()
with condition:
    print("condition acquired")
    
def waiter():
    print(threading.main_thread())
    print(threading.current_thread())
    with condition:
        print("Block on waiting for condition")
        condition.wait()
        print("condition met")
def busboy():
    print(threading.main_thread())
    print(threading.current_thread())
    with condition:
        condition.notify()
        
thread_waiter = Thread(target=waiter)
thread_waiter.start()
thread_busboy = Thread(target=busboy)
thread_busboy.start()

condition acquired
<_MainThread(MainThread, started 29136)>
<Thread(Thread-7 (waiter), started 33300)>
Block on waiting for condition
<_MainThread(MainThread, started 29136)>
<Thread(Thread-8 (busboy), started 36824)>
condition met


## Query Thread Attributes

In [7]:
def worker():
    pass

thread = Thread(target=worker)
print(thread.name)
print(thread.ident)
print(thread.daemon)
print(thread.native_id)
print(thread.is_alive())

# Enumerate active threads
print(threading.enumerate())

Thread-10 (worker)
None
False
None
False
[<_MainThread(MainThread, started 29136)>, <Thread(IOPub, started daemon 18944)>, <Heartbeat(Heartbeat, started daemon 35044)>, <ControlThread(Control, started daemon 20488)>, <HistorySavingThread(IPythonHistorySavingThread, started 36936)>, <ParentPollerWindows(Thread-4, started daemon 29312)>]


## Wait for a Result from a Thread

In [ ]:
# Use instance variables on an extend Thread

# Use a queue.Queue to share a result between threads

# Use a global variable to share a result between threads
